# PyTorch のコードを Sagemaker Training に適したコードに書き換える
* [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) の画像分類問題を解く

## 書き換え前のベースのコード
そのまま動くことを確認します。

In [ ]:
import torch
import torchvision

# データロード
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
valid_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
valid_data_loader = torch.utils.data.DataLoader(valid_data, batch_size=4,shuffle=False, num_workers=2)

#モデル定義
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 16, kernel_size=(3,3), stride=1, padding=(1,1)),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(16*32*32,10),
    torch.nn.Softmax(dim=1)
)

# 学習
def exec_epoch(loader, model, train_flg, optimizer, criterion):
    total_loss = 0.0
    correct = 0
    count = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        if train_flg:
            inputs, labels = torch.autograd.Variable(inputs), torch.autograd.Variable(labels)
            optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if train_flg:
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
        pred_y = outputs.argmax(axis=1)
        correct += sum(labels==pred_y)
        count += len(labels)
    total_loss /= (i+1)
    total_acc = 100 * correct / count
    if train_flg:
        print(f'train_loss: {total_loss:.3f} train_acc: {total_acc:.3f}%',end=' ')
    else:
        print(f'valid_loss: {total_loss:.3f} valid_acc: {total_acc:.3f}%')
    return model

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.00001)
for epoch in range(2):
    print(f'epoch: {epoch+1}',end=' ')
    model = exec_epoch(train_data_loader, model, True, optimizer, criterion)
    exec_epoch(valid_data_loader, model, False, optimizer, criterion)

# モデル保存
torch.save(model.state_dict(),'1.pth')

## 書き換え前のコードが SageMaker Training で動くことも確認
処理自体は動くけれども出来上がったモデルも揮発する

In [ ]:
!pygmentize ./src/2-2-1/train.py

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='./src/2-2-1/train.py',
    py_version='py38', 
    framework_version='1.10.0',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit()

## モデルを S3 に保存するように書き換え  

### (coding) トレーニングスクリプトの書き換え
以下のセルを実行(shift+enter)することで、`%%writefile` というマジックコマンドによって `%%writefile` の後に指定した場所にセルの内容が保存されます。モデルを S3 に保存するように書き換えて実行して、トレーニングジョブを動かしましょう。  
書き換え場所や書き換え内容がわからない方はヒントを 4 つ用意しましたので参照してください。(ヒントと書かれている部分をクリックするとヒントの内容が見えるようになります）  


<details><summary>▶ヒント1: 書き換え場所</summary><div>

モデルを指定ディレクトリに保存するとトレーニング完了時に Amazon S3 に自動で転送されます。`モデルの保存`をしている場所を書き換える必要があります。

</div></details>

<details><summary>▶ヒント2: モデルの保存先</summary><div>

トレーニング完了時に Amazon S3 に自動で転送するには環境変数で定義された指定ディレクトリに保存する必要があります。

</div></details>

<details><summary>▶ヒント3: 使用する環境変数</summary><div>

`SM_MODEL_DIR` と `SM_OUTPUT_DATA_DIR` で指定された値のディレクトリに保存するとトレーニング完了時に Amazon S3 に自動で転送します。今回のケースでより適しているのは？（どちらに保存しても問題なく動作はします）

</div></details>

<details><summary>▶ヒント4: 環境変数を Python のコードで取得するには？</summary><div>

`os` モジュールの `os.environ.get('{環境変数名}')` もしくは `os.getenv()` を使用するのが一般的です。  
`getenv` は `envrion.get`同じ（エイリアス）なので、どちらを使用しても良いです。  
 事前に `import os` する必要があることに注意してください  

</div></details>

<details><summary>▶ヒント5: ディレクトリとファイル名をつなげる良い方法</summary><div>

ディレクトリは末尾がスラッシュがつくのかつかないのかわからないケースがあり、パスの文字列を作るとき混乱します。  
例：`/opt/ml/code` なのか `/opt/ml/code/` なのか、そこに例えば `train.py` を指定するとき、単純に文字列連結してしまうと、  
`/opt/ml/codetrain.py`となってしまったり、`/opt/ml/code//train.py`となってしまったりしまいます。  
`os` モジュールの `os.path.join(DIRECTORY,FILE)`を使うと、スラッシュの有無を気にせずにパスをきれいに生成してくれます。

</div></details>

In [ ]:
%%writefile ./src/2-2-2/train.py
import torch
import torchvision

# データロード
train_data = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
valid_data = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
valid_data_loader = torch.utils.data.DataLoader(valid_data, batch_size=4,shuffle=False, num_workers=2)

#モデル定義
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 16, kernel_size=(3,3), stride=1, padding=(1,1)),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(16*32*32,10),
    torch.nn.Softmax(dim=1)
)

# 学習
def exec_epoch(loader, model, train_flg, optimizer, criterion):
    total_loss = 0.0
    correct = 0
    count = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        if train_flg:
            inputs, labels = torch.autograd.Variable(inputs), torch.autograd.Variable(labels)
            optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if train_flg:
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
        pred_y = outputs.argmax(axis=1)
        correct += sum(labels==pred_y)
        count += len(labels)
    total_loss /= (i+1)
    total_acc = 100 * correct / count
    if train_flg:
        print(f'train_loss: {total_loss:.3f} train_acc: {total_acc:.3f}%',end=' ')
    else:
        print(f'valid_loss: {total_loss:.3f} valid_acc: {total_acc:.3f}%')
    return model

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.00001)
for epoch in range(2):
    print(f'epoch: {epoch+1}',end=' ')
    model = exec_epoch(train_data_loader, model, True, optimizer, criterion)
    exec_epoch(valid_data_loader, model, False, optimizer, criterion)

# モデル保存
torch.save(model.state_dict(),'1.pth')

### トレーニングジョブの実行
上記で作成したトレーニングスクリプトを実行してみます。  
SageMaker Notebook を利用している場合は `instance_type='local'` とするとトレーニングジョブを Notebook インスタンスで実行できるため起動が早く試しやすいです。

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='./src/2-2-2/train.py',
    py_version='py38', 
    framework_version='1.10.0',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit()

### モデルの確認
モデルが S3 に転送されているかを確認するため、S3 からモデルをダウンロードしてロードしてみます。

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import torch
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 16, kernel_size=(3,3), stride=1, padding=(1,1)),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(16*32*32,10),
    torch.nn.Softmax(dim=1)
)
model.load_state_dict(torch.load('1.pth'))
print(model)

## トレーニングデータを S3 からトレーニングインスタンスに転送する

トレーニングジョブでアーティファクト（今回ではモデル）を S3 に連携できました。次はトレーニングデータをトレーニングスクリプトを実行前に Amazon S3 から連携させます。
### データを Amazon S3 に送り込む
以下コードを実行して、Amazon S3 にトレーニングデータを Amazon S3 にデータを送り込みます。  
データの形式は元が torch.Tensor のため、そのまま pt 形式で保存することとします。  
送り込んだ先の URI は `train_s3_uri`, `valid_s3_uri` に格納されます。

In [ ]:
import torchvision
import numpy as np
import torch
import sagemaker
import os
train_dir = './train'
valid_dir = './valid'
!mkdir -p {train_dir}
!mkdir -p {valid_dir}
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=len(train_data))
valid_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
valid_data_loader = torch.utils.data.DataLoader(valid_data, batch_size=len(valid_data))
train_data_loaded = next(iter(train_data_loader))
torch.save(train_data_loaded, os.path.join(train_dir, 'train.pt'))
valid_data_loaded = next(iter(valid_data_loader))
torch.save(valid_data_loaded, os.path.join(valid_dir, 'valid.pt'))
train_s3_uri = sagemaker.session.Session().upload_data(path=train_dir, bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/2-2/train')
valid_s3_uri = sagemaker.session.Session().upload_data(path=valid_dir, bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/2-2/valid')

### (coding) トレーニングコードの書き換え
`fit` メソッド実行時に引数に以下の引数を指定してトレーニングインスタンスにトレーニングデータを送り込みます。
`fit({'train':train_s3_uri,'valid':valid_s3_uri})`  
トレーニングインスタンスに送り込まれたデータを使ってトレーニングするようにトレーニングコードを書き換えましょう。


<details><summary>▶ヒント1: 書き換え場所</summary><div>

今までは torchvision のモジュールにある `torchvision.datasets.CIFAR10` でデータを読み込んでいました。  

`CIFAR10` 部分をファイルからロードするように書き換える必要があります。

</div></details>

<details><summary>▶ヒント2: ロードするファイルの場所</summary><div>

トレーニングスクリプトを実行する時のカレントディレクトリは `/opt/ml/code` であり、トレーニングスクリプトが配置された場所です。  
そこに `.npy` ファイルは存在しないため、パスを指定してファイルを読み込む必要があります。  
ファイルの場所は環境変数で定義された `SM_CHANNEL_{辞書形式で渡したfitの引数のキー名の大文字}` の値のディレクトリです。

</div></details>

<details><summary>▶ヒント3: .pt ファイルの読み込み方</summary><div>

` torch.utils.data.TensorDataset(*torch.load(FILEPATH))` で読み込めます。

</div></details>


In [ ]:
%%writefile ./src/2-2-3/train.py

import torch
import torchvision
import os

# データロード
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
valid_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
valid_data_loader = torch.utils.data.DataLoader(valid_data, batch_size=4,shuffle=False, num_workers=2)

#モデル定義
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 16, kernel_size=(3,3), stride=1, padding=(1,1)),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(16*32*32,10),
    torch.nn.Softmax(dim=1)
)

# 学習
def exec_epoch(loader, model, train_flg, optimizer, criterion):
    total_loss = 0.0
    correct = 0
    count = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        if train_flg:
            inputs, labels = torch.autograd.Variable(inputs), torch.autograd.Variable(labels)
            optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if train_flg:
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
        pred_y = outputs.argmax(axis=1)
        correct += sum(labels==pred_y)
        count += len(labels)
    total_loss /= (i+1)
    total_acc = 100 * correct / count
    if train_flg:
        print(f'train_loss: {total_loss:.3f} train_acc: {total_acc:.3f}%',end=' ')
    else:
        print(f'valid_loss: {total_loss:.3f} valid_acc: {total_acc:.3f}%')
    return model

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.00001)
for epoch in range(2):
    print(f'epoch: {epoch+1}',end=' ')
    model = exec_epoch(train_data_loader, model, True, optimizer, criterion)
    exec_epoch(valid_data_loader, model, False, optimizer, criterion)

# モデル保存
model_dir = os.environ.get('SM_MODEL_DIR')
torch.save(model.state_dict(),os.path.join(model_dir,'1.pth'))

### トレーニングジョブの実行

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='./src/2-2-3/train.py',
    py_version='py38', 
    framework_version='1.10.0',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit({
    'train':train_s3_uri,
    'valid':valid_s3_uri,
})

### モデルの確認

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import torch
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 16, kernel_size=(3,3), stride=1, padding=(1,1)),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(16*32*32,10),
    torch.nn.Softmax(dim=1)
)
model.load_state_dict(torch.load('1.pth'))
print(model)

### ハイパーパラメータを Estimator で定義する
`batch_size` や `epochs` などのパラメータはいろいろ試して良い数値を探索することがあります。  
毎回トレーニングコードを書き換えるのではなく外部からパラメータを設定できたほうが何かと便利です。  
SageMaker では Estimator インスタンス生成時の引数で `hyperparameters` という引数を用いることでトレーニングスクリプトの外からパラメータを与えることができます。  
ここでは以下のハイパーパラメータを与えたときにそのように動くようにトレーニングスクリプトを書き換えましょう。

```python
hyperparameters={
    'filters': 8,
    'epochs': 1,
    'batch_size': 16,
    'learning_rate': 0.001
}
```
<details><summary>ヒント1: スクリプト実行時にどうやってハイパーパラメータがスクリプトに引き渡されるか</summary><div>

3 つあり、どれを使ってもよいです。それぞれ default 値を設定しておくと良いでしょう。

1. コマンドライン引数
  トレーニングスクリプトを実行する際に以下のように実行されます。
  `python train.py --filters 8 --epochs 1 --batch_size 16 --learning_rate 0.001 --model_dir s3://{bucket}/prefix`
  [argparse モジュール](https://docs.python.org/ja/3/howto/argparse.html)を使って受け取ると良いでしょう。
  ユーザが設定していないハイパーパラメータ `model_dir` (モデルが S3 のどこに保存されるか)が自動で設定されることに注意しましょう。
  default 値を設定する場合は、`add_argument()` を実行する際に、default 引数を設定します。

2. SM_HPS 環境変数
  SM_HPS という環境変数に json 文字列で格納されるので、それを取得するのもよいでしょう。  
  以下のように取得すると `hps` 変数に辞書形式で取得できます。  
  
  ```python
  import json, os
  hps = json.loads(os.environ.get('SM_HPS'))
  ```
  デフォルト値は `hps.setdefault(KEY,VALUE)`を利用すると良いでしょう。
  
3. SM_HP_{ハイパーパラメータ名大文字}
  `os.environ.get()`で取得できます。デフォルト値の設定には第2引数にデフォルト値を設定します。

</div></details>


In [ ]:
%%writefile ./src/2-2-4/train.py
import torch
import torchvision
import os

# データロード
train_dir = os.environ.get('SM_CHANNEL_TRAIN')
valid_dir = os.environ.get('SM_CHANNEL_VALID')
train_data = torch.utils.data.TensorDataset(*torch.load(os.path.join(train_dir, 'train.pt')))
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
valid_data = torch.utils.data.TensorDataset(*torch.load(os.path.join(valid_dir, 'valid.pt')))
valid_data_loader = torch.utils.data.DataLoader(valid_data, batch_size=4, shuffle=True, num_workers=2)

#モデル定義
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 16, kernel_size=(3,3), stride=1, padding=(1,1)),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(16*32*32,10),
    torch.nn.Softmax(dim=1)
)

# 学習
def exec_epoch(loader, model, train_flg, optimizer, criterion):
    total_loss = 0.0
    correct = 0
    count = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        if train_flg:
            inputs, labels = torch.autograd.Variable(inputs), torch.autograd.Variable(labels)
            optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if train_flg:
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
        pred_y = outputs.argmax(axis=1)
        correct += sum(labels==pred_y)
        count += len(labels)
    total_loss /= (i+1)
    total_acc = 100 * correct / count
    if train_flg:
        print(f'train_loss: {total_loss:.3f} train_acc: {total_acc:.3f}%',end=' ')
    else:
        print(f'valid_loss: {total_loss:.3f} valid_acc: {total_acc:.3f}%')
    return model

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.00001)
for epoch in range(2):
    print(f'epoch: {epoch+1}',end=' ')
    model = exec_epoch(train_data_loader, model, True, optimizer, criterion)
    exec_epoch(valid_data_loader, model, False, optimizer, criterion)

# モデル保存
model_dir = os.environ.get('SM_MODEL_DIR')
torch.save(model.state_dict(),os.path.join(model_dir,'1.pth'))

### トレーニングの実行

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='./src/2-2-4/train.py',
    py_version='py38', 
    framework_version='1.10.0',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'filters':8,
        'epochs':1,
        'batch-size':'16',
        'learning_rate' : 0.001
    }
)
estimator.fit({
    'train':train_s3_uri,
    'valid':valid_s3_uri,
})

### 出来上がったモデルの確認

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import torch
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 8, kernel_size=(3,3), stride=1, padding=(1,1)),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(8*32*32,10),
    torch.nn.Softmax(dim=1)
)
model.load_state_dict(torch.load('1.pth'))
print(model)